In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,    # 숫자가 높을수록 랜덤함.
    )    

# 사용비용을 알고 싶다면 이렇게 with 안에 넣고 사용해야 함.
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of yeast\n\nInstructions:\n1. Rinse the rice under cold water until the water runs clear.\n2. Cook the rice in a rice cooker or on the stovetop with 1 cup of water until it is soft and fully cooked.\n3. Let the cooked rice cool to room temperature.\n4. In a large bowl, mix the nuruk and yeast with the cooked rice.\n5. Cover the bowl with a clean cloth and let it sit at room temperature for 3-4 days, stirring occasionally.\n6. After 3-4 days, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.\n7. Transfer the liquid to a clean container and let it ferment for an additional 2-3 days.\n8. Once the soju has reache